# 자동미분

In [1]:
import tensorflow as tf
import numpy as np

### `tf.GradientTape`
- tf.GradientTape 는 context 안에서 실행된 모든 연산을 tape(테이프) 에 기록한다.
- 그 다음 텐서플로는 후진 방식 자동미준을 사용해 테이프에 기록된 연산의 그래디언트를 계산한다.
- 미분은 ML에서 최적화와 연관이 있다.

## 1) scalar를 scalar로 미분

In [2]:
x = tf.Variable(3.0)  # GradientTape은 Variable에 대한 미분값만 추적한다. x는 반드시 Variable 이다!

with tf.GradientTape() as tape:
    y = x**2

In [3]:
y

<tf.Tensor: shape=(), dtype=float32, numpy=9.0>

- y=9 이므로 정답!

In [4]:
# dy = 2x * dx

dy_dx = tape.gradient(y, x)
dy_dx.numpy()

6.0

## 2) scalar를 vector로 미분

In [5]:
w = tf.Variable(tf.random.normal((3, 2)), name='w')
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1., 2., 3.]]

with tf.GradientTape(persistent=True) as tape:  # persistent=True 가 없으면 2번이상 호출 불가능
    y = x @ w + b
    loss = tf.reduce_mean(y**2)

In [6]:
[dl_dw, dl_db] = tape.gradient(loss, [w, b]) # list로 입력
[dl_dw, dl_db]

[<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[1.3279841, 2.3117962],
        [2.6559682, 4.6235924],
        [3.9839523, 6.9353886]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.3279841, 2.3117962], dtype=float32)>]

In [7]:
my_vars = {"w":w, "b":b}
grad = tape.gradient(loss, my_vars)  # dic으로 입력
grad['b']

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.3279841, 2.3117962], dtype=float32)>

## 3) 자동미분 컨트롤하기
- `tf.Variable`만 기록한다
- variable + tensor는 tensor를 반환
- `trainable` 조건으로 미분 기록을 제어(variable의 기록을 끌 수 있음)

In [8]:
# trainable Variable
x0 = tf.Variable(3.0, name='x0')

# Not trainable 
x1 = tf.Variable(3.0, name='x1', trainable=False)

# Not Variable (variable + tensor 이기 때문)
x2 = tf.Variable(2.0, name='x2') + 1.0

# Not Variable
x3 = tf.constant(3.0, name='x3')

In [9]:
with tf.GradientTape() as tape:
    y = (x0**2) + (x1**2) + (x2**2)
    
grad = tape.gradient(y, [x0, x1, x2, x3])

for g in grad:
    print(g)

tf.Tensor(6.0, shape=(), dtype=float32)
None
None
None


- trainable Variable 인 x0 값만 출력되고 있다.

## 4) 기록되고 있는 variable 확인하기

In [10]:
tape.watched_variables()

(<tf.Variable 'x0:0' shape=() dtype=float32, numpy=3.0>,)